# File Reader

Copyright 2022 Michael George (AKA Logiqx).

This file is part of [GPS Wizard](https://logiqx.github.io/gps-wizard/) and is distributed under the terms of the GNU General Public License.

GPS Wizard is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

GPS Wizard is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with GPS Wizard. If not, see <https://www.gnu.org/licenses/>.

In [3]:
import sys
import os

import unittest

for path in ['python', '.', '..']:
    readersPath = os.path.join(path, 'readers')
    if readersPath not in sys.path:
        sys.path.extend([readersPath])

from fit_reader import FitReader
from gpx_reader import GpxReader
from nmea_reader import NmeaReader
from oao_reader import OaoReader
from sbn_reader import SbnReader
from sbp_reader import SbpReader
from ubx_reader import UbxReader

## Main Method

In [4]:
def getFileReader(filename):
    '''Determine file type and return appropriate file reader'''

    ext = os.path.splitext(filename)[1].lower()

    if ext == '.fit':
        fileReader = FitReader(filename)
    elif ext == '.gpx':
        fileReader = GpxReader(filename)
    elif ext in ('.nmea', '.txt'):
        fileReader = NmeaReader(filename)
    elif ext == '.oao':
        fileReader = OaoReader(filename)
    elif ext == '.sbn':
        fileReader = SbnReader(filename)
    elif ext == '.sbp':
        fileReader = SbpReader(filename)
    elif ext == '.ubx':
        fileReader = UbxReader(filename)
    else:
        raise RuntimeError('Extension {} is not supported'.format(ext))

    return fileReader

## Unit Tests

In [5]:
class TestReaders(unittest.TestCase):
    '''Class to test all of the file readers'''

    def testFitApexPro(self):
        '''Test loading of an FIT file from COROS Apex Pro'''

        filename = os.path.join(projdir, 'sessions', '20220411', 'APEX_Pro_Speedsurfing20220411111317.fit')
        fileReader = getFileReader(filename)
        fileReader.load()


    def testGpxApexPro(self):
        '''Test loading of an GPX file from COROS Apex Pro'''

        filename = os.path.join(projdir, 'sessions', '20220411', 'APEX_Pro_Speedsurfing20220411111317.gpx')
        fileReader = getFileReader(filename)
        fileReader.load()


    def testNmeaGt31(self):
        '''Test loading of an NMEA file from a GT-31'''

        filename = os.path.join(projdir, 'sessions', '20220411', 'GT31_1Hz_GEORG30MICHA_932000175_20220411_111600.nmea')
        fileReader = getFileReader(filename)
        fileReader.load()


    def testOaoMini(self):
        '''Test loading of an OAO file from a Motion Mini'''

        filename = os.path.join(projdir, 'sessions', '20220411', 'Motion_Mini_10Hz_0470_2022-04-11-1117.oao')
        fileReader = getFileReader(filename)
        fileReader.load()


    def testOaoEsp(self):
        '''Test loading of an OAO file from a ESP-GPS'''

        filename = os.path.join(projdir, 'sessions', '20211230-esp', 'Head_L_7C9EBDFAF5C8_007.oao')
        fileReader = getFileReader(filename)
        fileReader.load()


    def testSbnGt11(self):
        '''Test loading of an SBN file from a GT-11'''

        filename = os.path.join(projdir, 'sessions', '20071227', 'MIKE_G_1003053_20071227_133836.SBN')
        fileReader = getFileReader(filename)
        fileReader.load()


    def testSbnGt31(self):
        '''Test loading of an SBN file from a GT-31'''

        filename = os.path.join(projdir, 'sessions', '20220411', 'GT31_1Hz_GEORG30MICHA_932000175_20220411_111600.SBN')
        fileReader = getFileReader(filename)
        fileReader.load()


    def testSbpGt11(self):
        '''Test loading of an SBP file from a GT-11'''

        filename = os.path.join(projdir, 'sessions', '20071227', 'MIKE_G_1003053_20071227_165512_DLG.SBP')
        fileReader = getFileReader(filename)
        fileReader.load()


    def testSbpGt31(self):
        '''Test loading of an SBP file from a GT-31'''

        filename = os.path.join(projdir, 'sessions', '20220411', 'GT31_1Hz_GEORG30MICHA_932000175_20220512_094254_DLG.SBP')
        fileReader = getFileReader(filename)
        fileReader.load()


    def testUbxEsp(self):
        '''Test loading of a UBX file from an ESP-GPS'''

        filename = os.path.join(projdir, 'sessions', '20211230-esp', 'Head_L_7C9EBDFAF5C8_007.ubx')
        fileReader = getFileReader(filename)
        fileReader.load()

In [6]:
if __name__ == '__main__':
    # Determine whether session is interactive or batch to facilitate unittest.main(..., exit=testExit)
    import __main__ as main
    testExit = hasattr(main, '__file__')

    projdir = os.path.realpath(os.path.join(sys.path[0], "..", ".."))

    unittest.main(argv=['first-arg-is-ignored'], exit=testExit)

..........
----------------------------------------------------------------------
Ran 10 tests in 2.544s

OK
